In [ ]:
import pandas as pd


In [12]:
df_long_metrics = pd.read_csv("df_long_byturn_v5 (1).csv")

In [ ]:
df_user = df_long_metrics[df_long_metrics['role'] == 'user'].copy()
df_model = df_long_metrics[df_long_metrics['role'] == 'model'].copy()

df_user.rename(columns={'turn': 'turn_user',
                          'text': 'text_user',
                          'flesch_reading_ease': 'flesch_reading_ease_user',
                          'flesch_kincaid_grade': 'flesch_kincaid_grade_user',
                          'tb_polarity': 'tb_polarity_user',
                          'tb_subjectivity': 'tb_subjectivity_user'}, inplace=True)

df_model.rename(columns={'turn': 'turn_model',
                           'text': 'text_model',
                           'flesch_reading_ease': 'flesch_reading_ease_model',
                           'flesch_kincaid_grade': 'flesch_kincaid_grade_model',
                           'tb_polarity': 'tb_polarity_model',
                           'tb_subjectivity': 'tb_subjectivity_model'}, inplace=True)

df_model['turn_model_plus1'] = df_model['turn_model'] + 1

# Merge USER turn t with MODEL turn t-1
df_merged_user_pred = pd.merge(
    df_user,
    df_model,
    left_on=['conversation_id', 'turn_user'], 
    right_on=['conversation_id', 'turn_model_plus1'],
    how='inner'
)


print("Columns in df_merged_user_pred:", df_merged_user_pred.columns.tolist())
print(df_merged_user_pred.head())

Columns in df_merged_user_pred: ['conversation_id', 'turn_user', 'role_x', 'text_user', 'flesch_reading_ease_user', 'flesch_kincaid_grade_user', 'tb_polarity_user', 'tb_subjectivity_user', 'turn_model', 'role_y', 'text_model', 'flesch_reading_ease_model', 'flesch_kincaid_grade_model', 'tb_polarity_model', 'tb_subjectivity_model', 'turn_model_plus1']
  conversation_id  turn_user role_x  \
0           c1000          1   user   
1           c1000          1   user   
2           c1000          1   user   
3           c1000          1   user   
4           c1000          2   user   

                                           text_user  \
0  I would like assistance in understanding how I...   
1  I would like assistance in understanding how I...   
2  I would like assistance in understanding how I...   
3  I would like assistance in understanding how I...   
4  Can you please break it down in a summary format?   

   flesch_reading_ease_user  flesch_kincaid_grade_user  tb_polarity_user  \


In [14]:
import statsmodels.formula.api as smf


df_analysis = df_merged_user_pred.dropna(subset=[
    'tb_polarity_user',
    'tb_polarity_model'
]).copy()

formula_user_pol = "tb_polarity_user ~ tb_polarity_model"
model_user_pol = smf.ols(formula=formula_user_pol, data=df_analysis).fit()
print("--- Predicting User Polarity from Previous Model Polarity ---")
print(model_user_pol.summary())
print("\n" + "="*80 + "\n")



df_analysis = df_merged_user_pred.dropna(subset=[
    'tb_subjectivity_user',
    'tb_subjectivity_model'
]).copy()

formula_user_subj = "tb_subjectivity_user ~ tb_subjectivity_model"
model_user_subj = smf.ols(formula=formula_user_subj, data=df_analysis).fit()
print("--- Predicting User Subjectivity from Previous Model Subjectivity ---")
print(model_user_subj.summary())
print("\n" + "="*80 + "\n")



--- Predicting User Polarity from Previous Model Polarity ---
                            OLS Regression Results                            
Dep. Variable:       tb_polarity_user   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     15.82
Date:                Tue, 08 Apr 2025   Prob (F-statistic):           7.23e-05
Time:                        00:29:13   Log-Likelihood:                -353.64
No. Observations:                1812   AIC:                             711.3
Df Residuals:                    1810   BIC:                             722.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------